In [1]:
import sys, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal
import importlib
import h5py
sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
outfile='/Users/ajc/Core/PostProcessing/2020/barrow_watershed/projections_02/Nov10/'

In [3]:
path_cades = "/Users/ajc/FUSE/simulations/production/barrow_watershed"
path = "/Users/ajc/Core/simulations/projections/barrow_watershed"

var_set = ['surface_flux-day.dat']

In [45]:
S1 = [os.path.join(path, 'CESM_16/CESM_5050_R1%s_BA'%i) for i in range(1)] 
S2 = [os.path.join(path, 'CESM_16/CESM_5050_R2%s_B'%i) for i in range(1)] 
S3 = [os.path.join(path, 'CESM_16/CESM_5050_R3%s_B'%i) for i in range(1)] 
S4 = [os.path.join(path, 'CESM_16/CESM_5050_R4%s_B'%i) for i in range(1)] 
S6 = [os.path.join(path, 'CESM_16/CESM_5050_R6%s_BA'%i) for i in range(1)]

#Simulations = [S1, S2, S3, S4,S6]
Simulations = [S1, S6]

#S1 = [os.path.join(path, 'CESM_16/trash')] 
#Simulations = [S1]
print (Simulations[0],)

['/Users/ajc/Core/simulations/projections/barrow_watershed/CESM_16/CESM_5050_R10_BA']


In [46]:
def read_file(file):
    d = np.loadtxt(file,usecols=1,skiprows=6)
    t1 = np.loadtxt(file,usecols=0,skiprows=6)
    t = np.array([x/86400./365. for x in t1])
    #print (len(d),len(t))
    l = np.minimum(len(d),len(t))
    return d[:l],t[:l]

def read_set(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path,s, var)
        #print (file)
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        t = np.array([x/86400./365. for x in t1])
        
        L = len(d)%365
        if L >0:
            d = d[:-L]
            t = t[:len(d)]
            
        dat.append(d)
        tim.append(t)
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    print ('Lengths: ',len(dat)%365, len(tim)%365,L)
    
    return dat, tim

In [47]:
path_sf ="/Users/ajc/Core/simulations/projections/barrow_watershed/CESM_15/CESM_5050_R60_B/visdump_surface_star_data.h5"
file_surf= h5py.File(path_sf,'r')
#print (file_surf.keys())
#print (file_surf['surface_star-cell_volume.cell.0'].keys())
basin_area = np.sum(list(file_surf['surface_star-cell_volume.cell.0']['100095']))
print (basin_area)

112635.47903126487


In [48]:
#forcing = h5py.File("/Users/ajc/FUSE/simulations/production/barrow_watershed/data/CESM/CESM-RCP8_5-2006-2100_snowadjusted_supersmoothDM_2006.h5",'r')
forcing_watershed_FM = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-FM_2006.h5",'r')

In [49]:
d1 = forcing_watershed_FM['precipitation rain [m s^-1]']
d2 = forcing_watershed_FM['precipitation snow [m SWE s^-1]']
rain_mm = np.reshape(d1,(-1,365)).mean(axis=1)*86400*365*1000
snow_mm = np.reshape(d2,(-1,365)).mean(axis=1)*86400*365*1000
precip_annual = snow_mm + rain_mm
r1 = np.array(d1)*86400*1000
r2 = np.array(d2)*86400*1000
precip_daily = r1 + r2

In [50]:
key_var = ['surface_star-evaporative_flux','surface_star-depth','surface_star-carbon_decomposition_rate']

In [51]:

def getSurfaceData(i,var,average=True):
    Data = []
    print ('getSurfaceData: ',var,average)
    for s in Simulations[i]:
        infile = os.path.join(s,'visdump_surface_star_data.h5')
        #print (infile)
        dx=h5py.File(infile,'r')
        #print (dx.keys())
        d = np.array(readdata.GetSurfVarFromVis_All(infile,var)[var])
        if (average):
            d = d.mean(axis=1)
        else:
            d = d.sum(axis=1)
        L = len(d)%365
        if L >0:
            d = d[:-L]
            
        Data.append(d)
        #print (d.shape, len(d), L)
    d = dict()
    d[var] = np.concatenate(Data)
    
    return d

In [52]:
#S1_Evap0 = getSurfaceData(0,key_var[0])

In [53]:
#S1_Evap = getSurfaceData(1)

In [54]:
def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[i:m])
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [55]:
colors = ['r','k','g','m','c','y']
marks = ['.','*', '+', 'o']
marks = ['-','-','-']
#Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$',r'S$_{4}$',r'S$_{6}$']
#Labels = ['basecase','basecase-subsidence','static_sg','static_sg-subsidence','dynamic_sg', 'dynamic_sg (dst=0.2)']
Labels = ['basecase','dynamic_sg']

In [56]:
#Slabels = ['S1', 'S2','S3','S4','S6','S6X']
Slabels = ['S1', 'S6']
#Labels = ['S1_press', 'S3_hybrid', 'S3_press']
SimData = dict()
for i,sim in enumerate(Simulations):
    var = var_set[0]
    d1, t1 = read_set(sim,var)
    SimData[Slabels[i] + 'time'] = t1
    SimData[Slabels[i] + 'data'] = d2

Lengths:  0 0 1
Lengths:  0 0 1


In [57]:
#Read discharge
Q = dict()
time = 2006 + np.linspace(0,94,94,dtype='int')
to_day_mm = 86400 * 1000
to_year_mm = 86400 * 365 * 1000
surface_area = 112635.48
to_mm = 1000.
for i,sim in enumerate(Simulations):
    var = var_set[0]
    
    d1, t1 = read_set(sim,var)
    d1 = [x/(55555.*surface_area)*to_day_mm for x in d1]
    
    d2 = np.reshape(d1,(-1,365)).mean(axis=1)*365
    t2 = np.reshape(t1,(-1,365)).mean(axis=1)
    #print (np.max(d2), len(d2), len(d1))
    d2 = get_moving_average(d2)
    Q[Slabels[i] + '_time'] = np.array(t2)
    Q[Slabels[i] + '_data'] = np.array(d2)

Lengths:  0 0 1
Lengths:  0 0 1


In [58]:
def plot_Q():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r','m']
 
    axs.fill_between(1985 + Q['S6_time'], Q['S6_data']-0, Q['S6_data']*0+Q['S1_data'] , color='lightgray')
    axs.fill_between(1985 + Q['S6_time'], Q['S3_data'], Q['S6_data']+0 , color='gray')
    
    axs.plot(1985 + Q['S6_time'],Q['S6_data'],color=Colors[i],label=Labels[i],linestyle='dashed')
    axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
    axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')
    axs.legend(loc='upper right',fontsize=8)
    #axs.set_xlim(2005,2100)
    #axs.plot(time,precip_annual[:-1],'g')


    #axs.plot(t1[tt0*365:(tt0+1)*365]-21,precip_daily[tt0*365:(tt0+1)*365],'g')
    #plt.plot(rain_mm,'k')
    #plt.plot(snow_mm,'m')
    #plt.plot(precip_annual,'r')
    #print (outfile)
    #plt.savefig(outfile+'discharge-annualavg_bars.pdf', bbox_inches='tight', dpi=500)
    #print (outfile+'discharge-annualavg_bars.pdf')

In [59]:
#plot_Q()

In [60]:

def get_Discharge():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    Colors = ['k','b','grey','g','r','m']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000

    to_mm = 1000.
    Discharge = dict()
    for i,sim in enumerate(Simulations):
        var = var_set[0]

        d1, t1 = read_set(sim,var)
        #print (d1)
        d1 = [x/(55250.*surface_area)*to_day_mm for x in d1] #converted to day here

        d2 = np.reshape(d1,(-1,365)).mean(axis=1)*365
        t2 = np.reshape(t1,(-1,365)).mean(axis=1)
        Discharge[Labels[i]] = d2
        #print ( len(d2), len(d1),d1)
        #d2 = get_moving_average(d2)

        axs.plot(1986 + t2,d2,color=Colors[i],label=Labels[i],linestyle='dashed')
        axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')
        axs.legend(loc='best',fontsize=8)
    return Discharge, t1

In [61]:
Q_annual, Time = get_Discharge()
print (len(Time),Q_annual)

Lengths:  0 0 1
Lengths:  0 0 1
730 {'basecase': array([42.65477033, 59.90851364]), 'dynamic_sg': array([10.71990068,  8.79278167])}


In [62]:
def Evaporation():
    %matplotlib qt
    fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r','m']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_day_year_mm = 86400 * 365 * 1000
    
    to_mm = 1000.
    Evap = dict()
    for i,sim in enumerate(Simulations):
        var = key_var[0]
        d1 = getSurfaceData(i, var)[var]
        l = len(d1)%365
        print (l)
        d3 = np.reshape(d1,(-1,365)).mean(axis=1)*to_day_year_mm
        Evap[Labels[i]] = d3
        #d3 = get_moving_average(d3)
        #print (d3)
        axs.plot(d3,color=Colors[i],label=Labels[i], linestyle="dashed")
        axs.legend(fontsize=8)
        
    return Evap

In [63]:
E_annual = Evaporation()
print (E_annual)

getSurfaceData:  surface_star-evaporative_flux True
0
getSurfaceData:  surface_star-evaporative_flux True
0
{'basecase': array([178.2485481 , 214.21704248]), 'dynamic_sg': array([201.87848976, 260.47267692])}


In [64]:
def get_water_content():
    Colors = ['k','b','grey','g','r']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000
    basin_area = 112635.48
    to_mm = 1000.
    m3_mm = 1e+9
    SS_change = dict()
    SF_change = dict()
    SWE_change = dict()
    for i,sim in enumerate(Simulations):
        var_wc = 'surface_star-column_water_content'
        var_wc_sf = 'surface_star-water_content'
        var_swe = 'surface_star-snow_water_equivalent'
        d1 = getSurfaceData(i, var_wc,average=False)[var_wc]
        d1_sf = getSurfaceData(i, var_wc_sf,average=False)[var_wc_sf]
        d1_swe = getSurfaceData(i, var_swe,average=False)[var_swe]
        
        d2 = np.reshape(d1,(-1,365))
        d2_sf = np.reshape(d1_sf,(-1,365))
        d2_swe = np.reshape(d1_swe,(-1,365))
        mol_density = 55300.
        
        factor = 1/(mol_density *surface_area)*to_mm
        SS_change[Labels[i]] = np.array([(d2[0][-1] - d2[0][0])*factor, (d2[1][-1] - d2[1][0])*factor])
        SF_change[Labels[i]] = np.array([(d2_sf[0][-1] - d2_sf[0][0])*factor,(d2_sf[1][-1] - d2_sf[1][0])*factor])
        SWE_change[Labels[i]] = np.array([(d2_swe[0][-1] - d2_swe[0][0])*to_mm/surface_area,(d2_swe[1][-1] - d2_swe[1][0])*to_mm/surface_area])
        print (Labels[i],d2_swe[0][0]*to_mm/surface_area, d2_swe[0][-1]*to_mm/surface_area)
    return SS_change, SF_change, SWE_change

In [65]:
SS_change, SF_change, SWE_change = get_water_content()

getSurfaceData:  surface_star-column_water_content False
getSurfaceData:  surface_star-water_content False
getSurfaceData:  surface_star-snow_water_equivalent False
basecase 44.07127691986398 72.97124802513012
getSurfaceData:  surface_star-column_water_content False
getSurfaceData:  surface_star-water_content False
getSurfaceData:  surface_star-snow_water_equivalent False
dynamic_sg 44.07127691986398 92.83561417767844


In [66]:
print (SS_change)
print(SF_change)
print(SWE_change)
print (E_annual)
print (Q_annual)

{'basecase': array([ 5.39120355, 16.2186138 ]), 'dynamic_sg': array([-1.84162869,  6.57765977])}
{'basecase': array([0.        , 0.96196841]), 'dynamic_sg': array([0.        , 0.01549338])}
{'basecase': array([ 28.89997111, -23.37663865]), 'dynamic_sg': array([ 48.76433726, -28.5255792 ])}
{'basecase': array([178.2485481 , 214.21704248]), 'dynamic_sg': array([201.87848976, 260.47267692])}
{'basecase': array([42.65477033, 59.90851364]), 'dynamic_sg': array([10.71990068,  8.79278167])}


In [75]:
MassBalanceB = []
MassBalanceD = []
for i in range(len(SS_change)):
    x = SS_change['basecase'][i] + SF_change['basecase'][i] + E_annual['basecase'][i] + Q_annual['basecase'][i]  + SWE_change['basecase'][i] 
    MassBalanceB.append(x)
    print ('Base:', np.round(precip_annual[i],2), np.round(x,2), ', SS: ', np.round(SS_change['basecase'][i],2) , 
           ', SF: ', np.round(SF_change['basecase'][i],2) ,', SWE: ', np.round(SWE_change['basecase'][i],2),
           ', Q: ',np.round(Q_annual['basecase'][i],2), ', E: ',np.round(E_annual['basecase'][i],2) )

    x = SS_change['dynamic_sg'][i] + SF_change['dynamic_sg'][i] + E_annual['dynamic_sg'][i] + Q_annual['dynamic_sg'][i]  + SWE_change['dynamic_sg'][i] 
    MassBalanceD.append(x)
    print ('D_sg:', np.round(precip_annual[i],2), np.round(x,2), ', SS: ',np.round(SS_change['dynamic_sg'][i],2) , 
           ', SF: ', np.round(SF_change['dynamic_sg'][i],2) , ', SWE: ', np.round(SWE_change['dynamic_sg'][i],2) , 
           ', Q: ',np.round(Q_annual['dynamic_sg'][i],2), ', E: ',np.round(E_annual['dynamic_sg'][i],2))
    print ('---------------------------------')
#print (MassBalanceB, MassBalanceD)
plt.plot(MassBalanceB,'k*-',label='basecase')
plt.plot(precip_annual[:2],'r*-',label='precip')
plt.plot(MassBalanceD,'g*-',label='dyn_sg')

plt.legend(loc='best',fontsize=8)

plt.ylim(240,270)

Base: 259.0 255.19 , SS:  5.39 , SF:  0.0 , SWE:  28.9 , Q:  42.65 , E:  178.25
D_sg: 259.0 259.52 , SS:  -1.84 , SF:  0.0 , SWE:  48.76 , Q:  10.72 , E:  201.88
---------------------------------
Base: 240.96 267.93 , SS:  16.22 , SF:  0.96 , SWE:  -23.38 , Q:  59.91 , E:  214.22
D_sg: 240.96 247.33 , SS:  6.58 , SF:  0.02 , SWE:  -28.53 , Q:  8.79 , E:  260.47
---------------------------------


(240, 270)

In [ ]:
For closed boundary:
Base: 259.0 262.14 7.11 0.01 28.97 0.0 226.05
D_sg: 259.0 259.35 -1.36 0.0 48.8 0.0 211.91
---------------------------------
Base: 240.96 249.86 14.48 1.15 -24.6 0.0 258.83
D_sg: 240.96 247.73 6.21 0.01 -28.0 0.0 269.52
---------------------------------

In [ ]:
def read_set_cycle(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path,s, var)
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        q = []
        t2 = []
        
        indx = np.where(t1%86400 == 43200.)[0]
        for i,x in enumerate(indx):
            if i ==0:
                q.append(0)
                t2.append(t1[0])
            else:
                sum1 =  np.mean(d[indx[i-1]:indx[i]])
                q.append(sum1)
                t2.append(t1[indx[i-1]])
                
        t = np.array([x/86400./365. for x in t2])
        
        d = q
        L = len(d)%365
        if L >0:
            d = d[:-L]
            t = t[:len(d)]
            
        dat.append(d)
        tim.append(t)
        print ('B:',len(t), len(d), L)
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    
    return dat, tim

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
    
plt.tight_layout()
#axs = axs.ravel()
Colors = ['k','b','grey','g','r','m']
time = 2006 + np.linspace(0,94,94,dtype='int')
to_day_mm = 86400 * 1000
to_year_mm = 86400 * 365 * 1000
basin_area = 112635.48
to_mm = 1000.
Discharge = dict()
for i,sim in enumerate(Simulations[:1]):
    d1, t1 = read_set_cycle(sim,'surface_flux-cycle.dat')
    d1 = [x/(55555.*basin_area)*to_day_mm for x in d1]
    
    #d2 = np.reshape(d1,(-1,365)).mean(axis=1)*365
    #t2 = np.reshape(t1,(-1,365)).mean(axis=1)
    
    d2 = np.reshape(d1,(-1,365))[0]#.mean(axis=1)*365
    t2 = np.reshape(t1,(-1,365))[0]#.mean(axis=1)
    
    d1A, t1A = read_set(sim,'surface_flux-day.dat')
    d1A = [x/(55555.*basin_area)*to_day_mm for x in d1A]
    
    #d2A = np.reshape(d1A,(-1,365)).mean(axis=1)*365
    #t2A = np.reshape(t1A,(-1,365)).mean(axis=1)
    
    d2A = np.reshape(d1A,(-1,365))[0]#.mean(axis=1)*365
    t2A = np.reshape(t1A,(-1,365))[0]#.mean(axis=1)
    
    axs.plot(1985 + t2,d2,color=Colors[i],label=Labels[i] + '-day',linestyle='solid')
    axs.plot(1985 + t2A,d2A,color='r',label=Labels[i] + '-cycle',linestyle='dotted')
    axs.legend()
    #axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
    #axs.set_ylabel('Annual average discharge [mm]',fontsize=12, fontweight='normal')